# CNN Model Creation

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, activations
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras.losses import CategoricalCrossentropy
from keras.models import Sequential
from keras.datasets import cifar10
from sklearn.preprocessing import LabelBinarizer

/Users/justinbell/Desktop/Computer Science/Projects/ML Models/Object Classifier/.venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# Data Preparation and Processing
data = cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train / 255.0
X_test = X_test / 255.0

#Target transformation
output_encoder = LabelBinarizer()
y_train = output_encoder.fit_transform(y_train)
y_test = output_encoder.transform(y_test)

 # Shapes of data variables
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

/Users/justinbell/Desktop/Computer Science/Projects/ML Models/Object Classifier/.venv/lib/python3.12/site-packages/keras/src/datasets/cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


X_train shape:  (50000, 32, 32, 3)
y_train shape:  (50000, 10)
X_test shape:  (10000, 32, 32, 3)
y_test shape:  (10000, 10)


In [3]:
print(output_encoder.classes_)

[0 1 2 3 4 5 6 7 8 9]


In [4]:
X_train.shape[1:4]

(32, 32, 3)

In [15]:
callbacks = []
epochs = 20
batch_size = 200
checkpoint = ModelCheckpoint(f"conv_weights/Conv2D Model 3/{epochs}e-{batch_size}bs" + "-{loss:.4f}" + ".keras", monitor='loss', verbose=1,
                                         save_best_only=True, mode='min')
callbacks = [checkpoint]

In [16]:
#Model Creation

model = Sequential([
    Conv2D(filters=32, kernel_size=(5,5), activation=activations.relu),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(filters=48, kernel_size=(7,7), activation=activations.relu),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(filters=100, kernel_size=(5,5), activation=activations.relu, padding='same'),
    Flatten(),
    Dense(units=50, activation=activations.relu),
    Dense(units=10, activation=activations.softmax)
])
model.build(input_shape=(None,) + X_train.shape[1:4])
print(model.summary())

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 28, 28, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 8, 8, 48)       │        75,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 4, 4, 100)      │       120,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 50)             │        80,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 278,404 (1.06 MB)

 Trainable params: 278,404 (1.06 MB)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
model.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])
# model.load_weights("conv_weights/Conv2D Model 1/50e-4000bs-0.8914.keras")
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=False, callbacks=callbacks, verbose=1, validation_split=0.1)

Epoch 1/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2849 - loss: 1.9402
Epoch 1: loss improved from None to 1.73720, saving model to conv_weights/Conv2D Model 3/20e-200bs-1.7372.keras

Epoch 1: finished saving model to conv_weights/Conv2D Model 3/20e-200bs-1.7372.keras
225/225 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.3615 - loss: 1.7372 - val_accuracy: 0.4568 - val_loss: 1.4675
Epoch 2/20
224/225 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4812 - loss: 1.4237
Epoch 2: loss improved from 1.73720 to 1.38557, saving model to conv_weights/Conv2D Model 3/20e-200bs-1.3856.keras

Epoch 2: finished saving model to conv_weights/Conv2D Model 3/20e-200bs-1.3856.keras
225/225 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.4972 - loss: 1.3856 - val_accuracy: 0.5490 - val_loss: 1.2704
Epoch 3/20
224/225 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5572 - loss: 1.2455
Epoch 3: loss improved from 1.38557 to 1.22804, saving model to conv_weights/Conv2D Model 3/20e-200bs-1

In [ ]:
predictions = model.predict(X_train[:10])
pred_decode = output_encoder.inverse_transform(predictions)
print(pred_decode)
print(output_encoder.inverse_transform(y_train[:10]))

In [ ]:
# import tensorflow as tf
# converter = tf.lite.TFLiteConverter.from_keras_model(model) # or .from_saved_model(saved_model_dir)
# tflite_model = converter.convert()
# with open('model.tflite', 'wb') as f:
#     f.write(tflite_model)

Saved artifact at '/var/folders/0_/nmh5w64x3q5_ffwfkr35sscw0000gn/T/tmphi4ywg6h'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor_43')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  4895227984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895229136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895229712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895229904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895231248: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1766688857.598078  212361 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1766688857.598484  212361 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
I0000 00:00:1766688857.608609  212361 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled


In [20]:
# Save the model to a .h5 file
model.save("object_classifier_model4.keras")

In [33]:
model.output_names = ['output']

In [ ]:
import coremltools as ct


# # Convert the model
# mlmodel = ct.convert(
#     'object_classifier_model4.h5',
#     convert_to="mlprogram", # Use mlprogram for better performance on newer iOS versions
#     minimum_deployment_target=ct.target.iOS15, # Specify your minimum iOS target
#     source="tensorflow"
# )

mlmodel = ct.converters.convert(model=model, 
                     inputs=[ct.TensorType(dtype=np.float16)], 
                     source="auto", convert_to="mlprogram", 
                     minimum_deployment_target=ct.target.iOS16)

# mlmodel = ct.convert(model=model, 
#                      inputs=[ct.TensorType(dtype=np.float16)], 
#                      source="tensorflow", convert_to="mlprogram", 
#                      minimum_deployment_target=ct.target.iOS16,
#                      output_names=['output'])
# Save the converted model
mlmodel.save("ObjectClassifierModel.mlpackage")